Дан датасет с результатами проведённых баскетбольных матчей. Необходимо сделать прогноз, сколько всего очков будет набрано обеими командами в матче.

Для каждой игры в датасете есть несколько строк, описывающие хронологию матча. В колонке "info" одинаковые записи соответствуют одному и тому же матчу.


Необходимо научить нейросеть предсказывать количество набранных очков в матче (fcount).

Добейтесь средней абсолютной ошибки 17 и менее очков.

### Подготовка  

In [ ]:
# Загрузка из google облака
import gdown
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l10/basketball.csv', None, quiet=True)

# Библиотека для работы с базами
import pandas as pd
df = pd.read_csv('basketball.csv', encoding= 'cp1251', sep=';', header=0, index_col=0) # Загружаем базу
df.head()

,TOTAL,info,Ком. 1,Ком. 2,Минута,Общая минута,Секунда,fcount,ftime
0,"98,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,0.0,1,1.0,30,81,90.0
1,"100,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,1,1.0,45,81,105.0
2,"99,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,2,2.0,0,81,120.0
3,"98,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,2,2.0,30,81,150.0
4,"95,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,3,3.0,0,81,180.0


Извлекаем текстовые данные из колонки `info` таблицы, помещаем в переменную `data_text`. Выводим длину списка:

In [ ]:
data_text = df['info'].values #

len(data_text) #

52450

Задаем максимальное кол-во слов в словаре, помещаем в переменную все символы, которые хотим вычистить из текста.

 Токенизируем текстовые данные:

In [ ]:
# Импортируем токенайзер
from tensorflow.keras.preprocessing.text import Tokenizer

maxWordsCount = 5000

sim_for_del='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'

tokenizer = Tokenizer (num_words=maxWordsCount,
                       filters=sim_for_del,
                       lower=True,
                       split=' ',
                       oov_token='unknown',
                       char_level=False)

tokenizer.fit_on_texts(data_text)

In [ ]:
# Переводим в Эмбеддинг пространство
Sequences = tokenizer.texts_to_sequences(data_text)

# Вариант  Bag of Words
xBOW_text = tokenizer.sequences_to_matrix(Sequences)

Преобразуем данные в numpy, подготовим наборы для обучения:

In [ ]:
# Библиотека работы с массивами
import numpy as np

xTrain = np.array(df[['Ком. 1','Ком. 2', 'Минута', 'Секунда','ftime']].astype('int'))
yTrain = np.array(df['fcount'].astype('int'))

In [ ]:
print(xTrain.shape)
print(yTrain.shape)
print(xBOW_text.shape)

(52450, 5)
(52450,)
(52450, 5000)


In [ ]:
import matplotlib.pyplot as plt

# Функция по проверке ошибки

def check_MAE_predictl_DubbleInput (model,
                                    x_data,
                                    x_data_text,
                                    y_data_not_scaled,
                                    plot=False):

  mae = 0 # Инициализируем начальное значение ошибки
  y_pred = (model.predict([x_data,x_data_text])).squeeze()

  for n in range (0,len(x_data)):
    mae += abs(y_data_not_scaled[n] - y_pred[n]) # Увеличиваем значение ошибки для текущего элемента
  mae /= len(x_data) # Считаем среднее значение
  print('Среднаяя абслолютная ошибка {:.3f} очков это {:.3f}% от общей выборки в {} игры'.format(mae, (mae/y_data_not_scaled.mean(axis=0))*100,len(x_data)))

  if plot:
     plt.scatter(y_data_not_scaled, y_pred)
     plt.xlabel('Правильные значение')
     plt.ylabel('Предсказания')
     plt.axis('equal')
     plt.xlim(plt.xlim())
     plt.ylim(plt.ylim())
     plt.plot([0, 250], [0, 250])
     plt.show()

In [ ]:
def on_epoch_end_custom(epoch, logs=None):
    check_MAE_predictl_DubbleInput(model_final_scaled,xTrain_scaled,xBOW_text,yTrain,plot=True)

In [ ]:
# Pешение

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, concatenate, Flatten
from tensorflow.keras.callbacks import LambdaCallback



In [ ]:
# Шаг 1: Подготовка данных

# Разделение на обучающую и тестовую выборки (80% - 20%)
from sklearn.model_selection import train_test_split

x_train, x_test, xBOW_train, xBOW_test, y_train, y_test = train_test_split(xTrain, xBOW_text, yTrain, test_size=0.2, random_state=42)

# Шаг 2: Определение модели

# Входные слои для числовых и текстовых данных
input_numeric = Input(shape=(x_train.shape[1],))
input_text = Input(shape=(xBOW_train.shape[1],))

# Скрытые слои для числовых данных
x1 = Dense(64, activation='relu')(input_numeric)
x1 = Dense(32, activation='relu')(x1)
x1 = Flatten()(x1)

# Скрытые слои для текстовых данных
x2 = Dense(64, activation='relu')(input_text)
x2 = Dense(32, activation='relu')(x2)

# Объединение обоих входов
combined = concatenate([x1, x2])

# Выходной слой
output = Dense(1)(combined)



In [ ]:
# Создание и компиляция модели
model = Model(inputs=[input_numeric, input_text], outputs=output)
model.compile(optimizer='adam', loss='mean_absolute_error')


In [ ]:
# Шаг 3: Обучение модели
epochs = 100
batch_size = 32



In [ ]:
# Колбэк для проверки MAE после каждой эпохи
on_epoch_end = LambdaCallback(on_epoch_end=lambda epoch, logs: check_MAE_predictl_DubbleInput(model, x_test, xBOW_test, y_test))

model.fit([x_train, xBOW_train], y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[on_epoch_end])



Epoch 1/100
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Среднаяя абслолютная ошибка 2.312 очков это 1.527% от общей выборки в 10490 игры
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 33.3313 - val_loss: 2.2851
Epoch 2/100
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Среднаяя абслолютная ошибка 2.154 очков это 1.423% от общей выборки в 10490 игры
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 2.1514 - val_loss: 2.1676
Epoch 3/100
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Среднаяя абслолютная ошибка 1.213 очков это 0.801% от общей выборки в 10490 игры
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 1.7346 - val_loss: 1.2593
Epoch 4/100
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Среднаяя абслолютная ошибка 1.210 очков это 0.799% от общей выборки в 10490 игры
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.6749 - val_loss: 1.2105
Epoch 5/100
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Среднаяя абслолютная ошибка 1.483 очков это 0.979% от общей выборки в 10490 игры
1049/1049 ━━━━━━━━━━━

In [ ]:
# Финальная оценка на тестовом наборе
check_MAE_predictl_DubbleInput(model, x_test, xBOW_test, y_test)

328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Среднаяя абслолютная ошибка 0.412 очков это 0.272% от общей выборки в 10490 игры
